In [20]:
from collections.abc import Sequence
import re
from itertools import combinations


In [24]:
def solve_a(lines: list[str]):
    pattern = re.compile("#+")
    arrangements = []
    for line in lines:
        these_arrangements = []
        row, numbers = line.split(" ")
        numbers_a = [int(num) for num in numbers.split(",")]
        q_indices = [i for i, char in enumerate(row) if char == "?"]
        num_missing_qs = sum(numbers_a) - row.count("#")
        for new_q_indices in combinations(q_indices, num_missing_qs):
            row_version = "".join(
                ["#" if i in new_q_indices else char for i, char in enumerate(row)]
            )
            blocks = pattern.findall(row_version)
            if [len(block) for block in blocks] == numbers_a:
                these_arrangements.append(new_q_indices)
        arrangements.append(these_arrangements)

    return arrangements

In [14]:
with open("./test.txt","r") as f:
    test_txt = f.read().splitlines()
    
with open("./input.txt","r") as f:
    input_txt = f.read().splitlines()

In [25]:
solutions = solve_a(input_txt)

In [26]:
solutions

[[(6, 9), (6, 10)],
 [(0, 2, 4),
  (0, 2, 6),
  (0, 2, 7),
  (0, 2, 9),
  (0, 2, 10),
  (0, 2, 11),
  (0, 3, 6),
  (0, 3, 7),
  (0, 3, 9),
  (0, 3, 10),
  (0, 3, 11),
  (0, 4, 6),
  (0, 4, 7),
  (0, 4, 9),
  (0, 4, 10),
  (0, 4, 11),
  (0, 6, 9),
  (0, 6, 10),
  (0, 6, 11),
  (0, 7, 9),
  (0, 7, 10),
  (0, 7, 11),
  (0, 9, 11),
  (1, 3, 6),
  (1, 3, 7),
  (1, 3, 9),
  (1, 3, 10),
  (1, 3, 11),
  (1, 4, 6),
  (1, 4, 7),
  (1, 4, 9),
  (1, 4, 10),
  (1, 4, 11),
  (1, 6, 9),
  (1, 6, 10),
  (1, 6, 11),
  (1, 7, 9),
  (1, 7, 10),
  (1, 7, 11),
  (1, 9, 11),
  (2, 4, 6),
  (2, 4, 7),
  (2, 4, 9),
  (2, 4, 10),
  (2, 4, 11),
  (2, 6, 9),
  (2, 6, 10),
  (2, 6, 11),
  (2, 7, 9),
  (2, 7, 10),
  (2, 7, 11),
  (2, 9, 11),
  (3, 6, 9),
  (3, 6, 10),
  (3, 6, 11),
  (3, 7, 9),
  (3, 7, 10),
  (3, 7, 11),
  (3, 9, 11),
  (4, 6, 9),
  (4, 6, 10),
  (4, 6, 11),
  (4, 7, 9),
  (4, 7, 10),
  (4, 7, 11),
  (4, 9, 11),
  (6, 9, 11),
  (7, 9, 11)],
 [(0, 1, 3, 7, 8),
  (0, 1, 4, 7, 8),
  (0, 1, 5, 7, 8),

In [21]:
class BranchingDict:
    def __init__(self):
        self.dict = {}

    def add(self, path: Sequence[int]):
        temp = self.dict
        for key in path[:-1]:
            temp = temp.setdefault(key, {})
        try:
            temp[path[-1]] = None
        except IndexError:
            print(path)
            exit()

    def get(self, path: Sequence[int]) -> dict | None:
        value = self.dict
        for key in path:
            value = value[key]
        return value

    def prune(self, path: Sequence[int]):
        if len(path) == 1:
            del self.dict[path[0]]
            return
        parent = self.get(path[:-1])
        del parent[path[-1]]  # type: ignore
        if parent == {}:
            self.prune(path[:-1])

    def get_a_path(self) -> list[int]:
        temp = self.dict
        path = []
        while temp is not None:
            key = list(temp.keys())[0]
            path.append(key)
            temp = temp[key]
        return path

In [124]:
def solve_line(line: str, pattern: re.Pattern) -> int:
    row, numbers = line.split(" ")
    numbers = [int(num) for num in numbers.split(",")]
    q_indices = [i for i, char in enumerate(row) if char == "?"]
    num_missing_qs = sum(numbers) - row.count("#")
    print(f"{num_missing_qs=}")
    solution_tree = BranchingDict()
    if num_missing_qs == 0:
        return [()]
    for combo in combinations(q_indices, num_missing_qs):
        solution_tree.add(combo)

    arrangements = []

    while solution_tree.dict:
        row_as_list = list(row)
        path = solution_tree.get_a_path()
        remaining_q_indices = [q for q in q_indices]
        valid_prefix = True
        string_so_far = ""
        blocks = []
        # check for early stopping
        if tuple(path) in solutions[i]:
            print("should be valid")
        else:
            print("shouldn't be")
        for path_idx in range(len(path)):
            row_idx = path[path_idx]
            row_as_list[row_idx] = "#"
            remaining_q_indices.remove(row_idx)
            for q_index in [i for i in remaining_q_indices if i < row_idx]:
                row_as_list[q_index] = "."
            string_so_far = "".join(row_as_list).split("?")[0]
            blocks = [len(block) for block in pattern.findall(string_so_far)]
            first_blocks = blocks[:-1]
            last_block = blocks[-1]

            if (first_blocks != numbers[: len(first_blocks)]):
                print("failed first cond")
                print(first_blocks)
                print(numbers[:len(first_blocks)])
            if len(numbers) < len(blocks):
                print("failed second cond")
            if last_block > numbers[len(blocks) - 1]:
                print("failed last cond")

            if (
                (first_blocks != numbers[: len(first_blocks)])
                or (len(numbers) < len(blocks))
                or (last_block > numbers[len(blocks) - 1])
            ):
                print(f"{path=}\n{''.join(row_as_list)} invalid for {numbers}")
                solution_tree.prune(path[: path_idx + 1])
                valid_prefix = False
                break
        if valid_prefix:
            if blocks == numbers:
                print(f"{path=}\n{''.join(row_as_list)} valid for {numbers}")
                arrangements.append(path)
            else:
                print(blocks)
                print(numbers)
                print(f"{path=}\n{''.join(row_as_list)} invalid at last sec for {numbers}")
            solution_tree.prune(path)
        print()

    return arrangements


In [76]:
def solve_b(lines: list[str]) -> list:
    pattern = re.compile("#+")
    arrangement_list = []
    for line in lines:
        arrangements = solve_line(line, pattern)
        arrangement_list.append(arrangements)

    return arrangement_list

In [55]:
solution_b = solve_b(input_txt)

In [102]:
for i in range(1000):
    a = set(tuple(i) for i in solutions[i])
    b = set(tuple(i) for i in solution_b[i])
    diff = a.difference(b)
    if diff:
        print(input_txt[i])
        print(diff)
        break
    

#???#?????#?#?? 2,2,1,1,1
{(1, 3, 8), (1, 5, 7), (1, 3, 7), (1, 3, 6), (1, 5, 8)}


In [82]:
s = list("#???#?????#?#??")
for i in (1,3,8):
    s[i] = "#"
print("".join(s))



##?##???#?#?#??


In [125]:
solve_line("#???#?????#?#?? 2,2,1,1,1", re.compile("[#9]+"))

num_missing_qs=3
shouldn't be
failed last cond
path=[1, 2, 3]
###?#?????#?#?? invalid for [2, 2, 1, 1, 1]

shouldn't be
failed last cond
path=[1, 3, 5]
##.###????#?#?? invalid for [2, 2, 1, 1, 1]

should be valid
[2, 2, 1]
[2, 2, 1, 1, 1]
path=[1, 3, 6]
##.##.#???#?#?? invalid at last sec for [2, 2, 1, 1, 1]

should be valid
[2, 2, 1]
[2, 2, 1, 1, 1]
path=[1, 3, 7]
##.##..#??#?#?? invalid at last sec for [2, 2, 1, 1, 1]

should be valid
[2, 2, 1]
[2, 2, 1, 1, 1]
path=[1, 3, 8]
##.##...#?#?#?? invalid at last sec for [2, 2, 1, 1, 1]

shouldn't be
failed last cond
path=[1, 3, 9]
##.##....##?#?? invalid for [2, 2, 1, 1, 1]

shouldn't be
failed last cond
path=[1, 3, 11]
##.##.....###?? invalid for [2, 2, 1, 1, 1]

shouldn't be
failed last cond
path=[1, 3, 13]
##.##.....#.##? invalid for [2, 2, 1, 1, 1]

should be valid
path=[1, 3, 14]
##.##.....#.#.# valid for [2, 2, 1, 1, 1]

shouldn't be
failed last cond
path=[1, 5, 6]
##..###???#?#?? invalid for [2, 2, 1, 1, 1]

should be valid
[2, 2, 1

[[1, 3, 14], [1, 5, 14]]